# Intro to PPL

To start: the "hello world" of statistics - coin flips with biased coin (where do you find one of these?).

In [ ]:
import numpy as np

Simulate biased coin flips and compute the likelihood. The coin flips are draws from Bernoulli distribution parameterized with h, the likelihood of the independent trials is:
$$
p(X\mid h) = successes^h failures^{1 - h}
$$

In [ ]:
def coin_flips(n_samples: int, p_heads: float):
    return np.cast[np.int32](np.random.uniform(size=n_samples) < p_heads)

def likelihood(samples, p_heads):
    n_heads = np.sum(samples == 1)
    return p_heads ** n_heads * (1 - p_heads) ** (len(samples) - n_heads)

In [ ]:
true_p_heads = 0.5
samples = coin_flips(n_samples=100, p_heads=true_p_heads)
print("samples:\n{}".format(samples))
print("heads: {}/{}, ratio: {}".format(np.sum(samples), len(samples), np.sum(samples) / len(samples)))

In [ ]:
%pylab inline

posterior:

$$
\begin{align}
p(H \mid X) &= \frac{p(X \mid H) p(H)}{p(X)}\\
&= \frac{p(X \mid H) p(H)}{\sum {p(X \mid H) p(H)}}
\end{align}
$$

In [ ]:
p_h = np.linspace(0.0, 1.0, num=100)
q_h = likelihood(samples=samples, p_heads=p_h)
q_h = q_h / np.sum(q_h)
plt.plot(p_h, q_h)
plt.vlines(x=true_p_heads, ymin=0.0, ymax=np.max(q_h), linestyle="dotted")
plt.show()

Let's introduce some more complex priors

In [ ]:
def prior(alpha, beta):
    return np.random.beta(a=alpha, b=beta)

In [ ]:
priors = [prior(alpha=20, beta=20) for _ in range(1000)]
plt.hist(priors, bins=20)
plt.xlim(0.0, 1.0)
plt.show();

# Enter tensorflow probability (which includes Edward2)

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import edward2 as ed

## Coin flips with Beta prior

In [ ]:
tf.reset_default_graph()

In [ ]:
def prior(alpha, beta):
    return ed.Beta(concentration0=alpha, concentration1=beta, name="prob")

def model(alpha, beta, size):
    p = prior(alpha=alpha, beta=beta)
    return ed.Bernoulli(probs=p * tf.ones(shape=size), name="outcomes")

In [ ]:
x_ = tf.placeholder(tf.float32)
pdf_prior = prior(alpha=20.0, beta=20.0).distribution.prob(value=x_)
n_samples_ = tf.placeholder(tf.int32)
samples = model(alpha=20.0, beta=20.0, size=n_samples_)

In [ ]:
x = np.linspace(0, 1.0, 100)
with tf.Session() as sess:
    samples_value, prior_pdf_value = sess.run([samples, pdf_prior], {x_: x, n_samples_: 100})

In [ ]:
plt.plot(x, prior_pdf_value);

In [ ]:
samples_value

Now, we will perform inference - find the posterior distribution of our hidden variable, which we called "prob".

First we will compute the log joint distribution of the model, which is needed by most of the inference algorithms.

In [ ]:
n_outcomes = 20
outcomes_ = tf.placeholder(tf.float32, shape=[n_outcomes])

In [ ]:
log_joint = ed.make_log_joint_fn(model)

The above line created a function that can compute the log joint distribution given values of all the model variables. As we are seeking to infer the distribution of "prob" we will explicitly condition on the observations by plugging in "outcomes_".

In [ ]:
def target_log_prob_fn(logit_prob):
    return log_joint(20.0, 20.0, outcomes_.shape[0], prob=tf.nn.sigmoid(logit_prob), outcomes=outcomes_)

The observations

In [ ]:
# outcomes = np.cast[np.int32](np.random.uniform(size=n_outcomes) < .1)
outcomes = np.zeros(n_outcomes)

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

Now some sampling ...

Using black box algorithm: Metropolis-Hastings, which performs random walk and accepts the state based on the ratio of the current and next state probabilities.

In [ ]:
mh_kernel = tfp.mcmc.MetropolisHastings(
    inner_kernel=tfp.mcmc.UncalibratedRandomWalk(target_log_prob_fn=target_log_prob_fn),
)
states, kernel_results = tfp.mcmc.sample_chain(
    num_results=1000,
    current_state=[.0],
    kernel=mh_kernel,
    num_burnin_steps=500)

In [ ]:
with tf.Session() as sess:
    s_, kr_ = sess.run([states, kernel_results], {
        outcomes_: outcomes
    })

In [ ]:
plt.hist(sigmoid(s_[0]), bins=20)
plt.xlim([0, 1.0])
plt.show();

What about Variational Inference? With the current state of Edward2 / TFP you need to manually define the ELBO loss and optimize it. The elegance of Edward just doesn't come through, hence I'll completely avoid it.

https://github.com/tensorflow/probability/blob/master/tensorflow_probability/python/edward2/Upgrading_From_Edward_To_Edward2.md#probabilistic-inference

## Bayesian linear regression

Let's step it up a notch with bayesian linear regression:

$$
y = a x + b + \epsilon \\
\epsilon \sim N(0, \sigma)
$$

Generate the data

In [ ]:
x = np.random.uniform(low=-10.0, high=10.0, size=20)
true_a = 0.5
true_b = 7
true_sigma = 1.0

y = np.random.normal(loc=true_a * x + true_b, scale=true_sigma)
plt.scatter(x, y, marker=".");

Define the model with explicit priors

In [ ]:
def linear_model(data):
    a = ed.Normal(loc=0.0, scale=1.0, name="a")
    b = ed.Normal(loc=6.0, scale=.5, name="b")
    mu = a * data + b
    return ed.Normal(loc=mu, scale=2.0, name="observations")

In [ ]:
log_joint = ed.make_log_joint_fn(linear_model)

def target_log_prob_fn(a, b):
    return log_joint(x, a=a, b=b, observations=y)

In [ ]:
mh_kernel = tfp.mcmc.MetropolisHastings(
    inner_kernel=tfp.mcmc.UncalibratedRandomWalk(target_log_prob_fn=target_log_prob_fn),
)
states, kernel_results = tfp.mcmc.sample_chain(
    num_results=1000,
    current_state=[.0, 2.0],
    kernel=mh_kernel,
    num_burnin_steps=500)

In [ ]:
with tf.Session() as sess:
    s_, kr_ = sess.run([states, kernel_results])

In [ ]:
plt.hist(s_[0], bins=10)
plt.show();
plt.hist(s_[1], bins=10)
plt.show();

In [ ]:
plt.scatter(x, y, marker=".")
full_x = np.linspace(-10, 10, 100)
for i in range(1000):
    pred_y = s_[0][i] * full_x + s_[1][i]
    plt.plot(full_x, pred_y, alpha=.01)

Bonus points: have $\sigma$ as random variable.